In [315]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import time
import scipy.optimize as optimize
import lightgbm as lgb
from bs4 import BeautifulSoup

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from scipy.stats import rankdata

from collections import defaultdict
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import TransformerMixin, BaseEstimator
from tqdm import tqdm

import time
import re 
import scipy
from scipy import sparse
import gc 
from IPython.display import display, HTML
from pprint import pprint
import warnings
warnings.filterwarnings("ignore")

In [3]:
system_path=r"C:\Users\Lenovo\Desktop\stupidcode\data\jigsaw"
#第一届 jigsaw比赛 数据（challenge） Toxic Comment Classification Challenge

jc_path=os.path.join(system_path,"jigsaw-toxic-comment-classification-challenge")
#ruddit 数据
run_path=os.path.join(system_path,"ruddit-jigsaw-dataset/Dataset")
#第二届 jigsaw比赛 对少数人群不歧视
juc_path=os.path.join(system_path,"jigsaw-unintended-bias-in-toxicity-classification")

#本次比赛数据 作为val
jts_path=os.path.join(system_path,"jigsaw-toxic-severity-rating")

## 翻译增强

In [ ]:
#正在加载...

In [316]:
jc_trans_fr=pd.read_csv(os.path.join(jc_path,"toxic_behavior_translation_fr.csv"))
jc_trans_fr=jc_trans_fr.rename(columns={'comment_text': 'text',"\r\n\r\nBacktranslated":"fr_text"})

In [317]:
jc_trans_es=pd.read_csv(os.path.join(jc_path,"toxic_behavior_translation_es.csv"))
jc_trans_es=jc_trans_es.rename(columns={'comment_text': 'text',"Backtranslated":"es_text"})

In [318]:
jc_trans_de=pd.read_csv(os.path.join(jc_path,"toxic_behavior_translation_de.csv"))
jc_trans_de=jc_trans_de.rename(columns={'comment_text': 'text',"\nBacktranslated":"de_text"})

jc_trans=pd.concat([jc_trans_fr,jc_trans_es["es_text"],jc_trans_de["de_text"]],axis=1)
jc_trans.set_index(["Unnamed: 0"], inplace=True)

jc_trans.to_csv(os.path.join(jc_path,"jc_trans.csv"))
###检测
# jc_trans_fr[jc_trans_fr["fr_text"]=="正在加载..."]

In [334]:
def text_cleaning(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string

    return text

In [319]:
##第四届数据

In [320]:
lesstoxic_fr=pd.read_csv(os.path.join(jts_path,"less_toxic_fr.csv"))
lesstoxic_fr=lesstoxic_fr.rename(columns={"Backtranslated":"less_toxic_fr"})
moretoxic_fr=pd.read_csv(os.path.join(jts_path,"more_toxic_fr.csv"))
moretoxic_fr=moretoxic_fr.rename(columns={"Backtranslated":"more_toxic_fr"})

lesstoxic_es=pd.read_csv(os.path.join(jts_path,"less_toxic_es.csv"))
lesstoxic_es=lesstoxic_es.rename(columns={"Backtranslated":"less_toxic_es"})
moretoxic_es=pd.read_csv(os.path.join(jts_path,"more_toxic_es.csv"))
moretoxic_es=moretoxic_es.rename(columns={"Backtranslated":"more_toxic_es"})

lesstoxic_de=pd.read_csv(os.path.join(jts_path,"less_toxic_de.csv"))
lesstoxic_de=lesstoxic_de.rename(columns={"\nBacktranslated":"less_toxic_de"})
moretoxic_de=pd.read_csv(os.path.join(jts_path,"more_toxic_de.csv"))
moretoxic_de=moretoxic_de.rename(columns={"Backtranslated":"more_toxic_de"})

# lesstoxic_de[lesstoxic_de["less_toxic_fr"]=="正在加载..."]

In [321]:
df_val = pd.read_csv(os.path.join(jts_path,"validation_data.csv"))

In [322]:
lesstoxic=pd.concat([lesstoxic_fr,lesstoxic_es["less_toxic_es"],lesstoxic_de["less_toxic_de"]],axis=1)
lesstoxic["less_toxic"]=df_val["less_toxic"]
moretoxic=pd.concat([moretoxic_fr,moretoxic_es["more_toxic_es"],moretoxic_de["more_toxic_de"]],axis=1)
moretoxic["more_toxic"]=df_val["more_toxic"]

validation_translation=pd.concat([lesstoxic,moretoxic],axis=1)
validation_translation.to_csv(os.path.join(jts_path,"validation_translation.csv"))
# only_trans=validation_translation[["less_toxic_fr","less_toxic_es","less_toxic_de","more_toxic_fr","more_toxic_fr","more_toxic_fr"]]
# validation_translation=pd.concat([df_val,only_trans],axis=1)


In [327]:
validation_translation.head()

,worker,less_toxic,less_toxic_fr,less_toxic_es,less_toxic_de,worker,more_toxic,more_toxic_fr,more_toxic_es,more_toxic_de
0,313,This article sucks \n\nwoo woo wooooooo,This article fears\n\nwoo woo woooooooo,This article stinks\n\nWoo Woo Wooooooo.,This article sucks off\n\nWoo Woo Wooooooo.,313,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...,WHAT !!!!!!!!?!? !!?!? !!?!?!?!? !!!!!!!!!!!!!...,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...","""And yes, people should recognize that, but th...","""And yes, people should recognize that, but th...","""And yes, people should recognize that, but th...",188,Daphne Guinness \n\nTop of the mornin' my fav...,Daphne Guinness\n\nTop of the Mornin 'my favo...,Daphne Guinness.\n\nCover the morning of my f...,Daphne Guinness.\n\nSummit of the morning my ...
2,82,"Western Media?\n\nYup, because every crime in...","Western media?\n\nYup, because every crime fr...","Western media?\n\nYes, because each crime aro...",Western media?\n\nThey because every crime is...,82,"""Atom you don't believe actual photos of mastu...","""Atom, you do not believe that the real photos...","""Atom that you do not believe that the real ph...","""Atom, you do not believe that the actual phot..."
3,347,And you removed it! You numbskull! I don't car...,And you deleted it! You NUMBSKULL! I do not ca...,And you took it out! Numbskull! I do not care ...,And you have removed it! You are Numbskull! I ...,347,You seem to have sand in your vagina.\n\nMight...,You seem to have sand in your vagina.\n\nMight...,It seems that you have sand in your vagina.\n\...,They seem to have sand in their vagina.\n\nMay...
4,539,smelly vagina \n\nBluerasberry why don't you ...,Smelly Vagina\n\nBlueRasberry Why do not you ...,"Skolly vagina.\n\nBluerasberry, why are not y...",stinking vagina\n\nBluerasberry Why is not sh...,539,"hey \n\nway to support nazis, you racist","Hey\n\nway of supporting the Nazis, you racist","Hears\n\nWay to support the Nazis, you are ra...","Hi\n\nWay to support Nazis, you racist"


In [210]:
lesstoxic=lesstoxic.rename(columns={"less_toxic":"text","less_toxic_fr":"text_fr","less_toxic_es":"text_es","less_toxic_de":"text_de"})
moretoxic=moretoxic.rename(columns={"more_toxic":"text","more_toxic_fr":"text_fr","more_toxic_es":"text_es","more_toxic_de":"text_de"})

comment_translation=pd.concat([lesstoxic,moretoxic])
comment_translation=comment_translation.drop_duplicates(["text"])

In [226]:
comment_translation.to_csv(os.path.join(jts_path,"comment_translation.csv"))

In [214]:
from sklearn.preprocessing import LabelEncoder
def generate_comments(data):
    more_toxic_text=data["more_toxic"].values
    less_toxic_text=data["less_toxic"].values    
    comments=np.concatenate((more_toxic_text,less_toxic_text))
    comments=np.unique(comments)
    comments=pd.DataFrame({"text":comments})
    text_encoder=LabelEncoder()
    text_encoder.fit(comments)
    comments["encode_text"]=text_encoder.transform(comments["text"])
    comments["toxic_value"]=0
    comments["access_time"]=0
    data["encode_less"]=text_encoder.transform(data["less_toxic"])
    data["encode_more"]=text_encoder.transform(data["more_toxic"])
    
    return data,comments

In [215]:
data,comments=generate_comments(df_val)

In [245]:
temp_translation=comment_translation
temp_translation=temp_translation.merge(comments,on="text",how="left")


In [246]:
temp_translation.head()

,worker,text,text_fr,text_es,text_de,encode_text,toxic_value,access_time
0,313,This article sucks \n\nwoo woo wooooooo,This article fears\n\nwoo woo woooooooo,This article stinks\n\nWoo Woo Wooooooo.,This article sucks off\n\nWoo Woo Wooooooo.,2405,0,0
1,188,"""And yes, people should recognize that but the...","""And yes, people should recognize that, but th...","""And yes, people should recognize that, but th...","""And yes, people should recognize that, but th...",7215,0,0
2,82,"Western Media?\n\nYup, because every crime in...","Western media?\n\nYup, because every crime fr...","Western media?\n\nYes, because each crime aro...",Western media?\n\nThey because every crime is...,2632,0,0
3,347,And you removed it! You numbskull! I don't car...,And you deleted it! You NUMBSKULL! I do not ca...,And you took it out! Numbskull! I do not care ...,And you have removed it! You are Numbskull! I ...,7973,0,0
4,539,smelly vagina \n\nBluerasberry why don't you ...,Smelly Vagina\n\nBlueRasberry Why do not you ...,"Skolly vagina.\n\nBluerasberry, why are not y...",stinking vagina\n\nBluerasberry Why is not sh...,3524,0,0


In [307]:
train=data[0:100]

In [308]:
train.head()

,worker,less_toxic,more_toxic,encode_less,encode_more
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...,2405,12151
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...,7215,653
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu...",2632,7222
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...,7973,12968
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist",3524,3266


In [309]:
temp_train=train.drop(["less_toxic","more_toxic"],axis=1)

In [310]:
temp_train=temp_train.merge(temp_translation[["encode_text","text_fr"]],left_on="encode_less",right_on="encode_text",how="left")
temp_train=temp_train.rename(columns={"text_fr":"less_toxic"})
temp_train.drop(["encode_text"],axis=1,inplace=True)

In [311]:
temp_train=temp_train.merge(temp_translation[["encode_text","text_fr"]],left_on="encode_more",right_on="encode_text",how="left")
temp_train=temp_train.rename(columns={"text_fr":"more_toxic"})
temp_train.drop(["encode_text"],axis=1,inplace=True)

In [312]:
train=pd.concat([train,temp_train])

In [314]:
train.shape

(200, 5)